# Baseline de algoritmos de xtracción de palabras clave

## Algoritmos básicos: TextRank y Fractalidad

Esta notebook está hecha para correr TextRank y Grado de Fractalidad sobre los textos con etiqueta 0 o etiqueta 5 comparando resultados con la base de datos del EmoPro

### Importes para librerías

In [2]:
from math import *
from math import sqrt
import string
# import csv
import operator
# import random
import pandas as pd
#librerias necesarias para text rank
from collections import OrderedDict
import numpy as np
import spacy
#import nlp

#Listado de STOPWORDS dependiendo del lenguaje
#from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.es.stop_words import STOP_WORDS

In [3]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 9.5 MB/s eta 0:00:000m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [4]:
nlp = spacy.load('es_core_news_sm')

## Definición de la clase TextRank

In [5]:
class TextRank4Keyword():
    """Extract keywords from text"""

    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 100 # iteration steps
        self.node_weight = None # save keywords and its weight


    def set_stopwords(self, stopwords):
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True

    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences

    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab

    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs

    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())

    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1

        # Get Symmeric matrix
        g = self.symmetrize(g)

        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm

        return g_norm


    def get_keywords(self, number=10):
        """Print top number keywords"""
        keysw={}
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            keysw[key] =value
            if i > number:
                break
        return keysw


    def analyze(self, text,
                candidate_pos=['NOUN', 'PROPN'],
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""

        # Set stop words
        self.set_stopwords(stopwords)

        # Pare text by spaCy
        doc = nlp(text)

        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words

        # Build vocabulary
        vocab = self.get_vocab(sentences)

        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)

        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)

        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))

        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]

        self.node_weight = node_weight


## Definición de la función Grado de Fractalidad

In [6]:
#solamente se calcula el grado de fractalidad de las palabras que tengan mas de uno de frecuencia
def fractalidad(palabras,vocabulario,frec,dist):
    N=len(palabras)                                     #El número de tokens de todo el texto
    gf={}
    cajas_index=set()
    voc=[]                                             #la variable voc contendra cada sintagma con frecuencia mayor que 1, por que las otras palabras tendrán 0 de grado de fractaldiad
    for p in vocabulario:                              #Esto se puede hacer fuera del algoritmo, pero se incluye para evitar ese calculo innecesario
        if(p not in voc):
            if(frec[p]>1):
                if(p not in STOP_WORDS):
                    if(len(p)>1):
                        voc.append(p)
    # print("Text size: ",N)
    # print("Vocabulary: ",len(voc))
    for p in voc:
        rcajas=dist[p]
        M=frec[p]
        dfw=0.0
        nsh=0.0
        for s in range(1,N+1):
            noc=0
            for e in rcajas:
                cajas_index.add(ceil(int(e)/s))
            noc=len(cajas_index)
            cajas_index.clear()
            ns=N/s
            if(M<=ns):
                nsh=M
            else:
                nsh=M/(1+(M-1)/(N-1)*(s-1))
            dfw=dfw+fabs(log(nsh/noc))
        gf[p]=dfw
    return gf    #regresamos un diccionario

In [7]:
def distribucion(palabras,vocabulario):
    N=len(palabras)
    ncajas=[]
    cajas={}
    frecuencias={}
    for p in vocabulario:
        ncajas.clear()
        i=0
        M=palabras.count(p)
        while(i<N):
            if(p == palabras[i]):
                ncajas.append(i+1)
            i=i+1
        frecuencias[p]=M
        cajas[p]=ncajas[:]
    return frecuencias,cajas

## Lectura de archivo de entrada

In [8]:
#Lectura de archivo para generación de vocabulario
def cargar_datos(filename):
    textopos = []
    textoneg =[]
    with open(filename, encoding="utf-8-sig") as f:
        texto = pd.read_csv(f)
        for _, r in texto.iterrows():
            if r['ds0'] == 1:
                textoneg.append(r['Texto'])
            elif r['ds5']==1:
                textopos.append(r['Texto'])
        for row in textopos:
            #Pasar a minusculas
            row = ' '.join(row)
            row = row.lower()
    #Eliminar puntuación
        for row in textoneg:
            row = ' '.join(row)
            row = row.lower()
    textopos =' '.join(textopos)
    textoneg=' '.join(textoneg)
    textopos = textopos.translate(str.maketrans('', '', string.punctuation))
    textoneg = textoneg.translate(str.maketrans('', '', string.punctuation))
    textopos = textopos.translate(str.maketrans('', '', '¿?¡!.:,;—“”0123456789’'))
    textoneg = textoneg.translate(str.maketrans('', '', '¿?¡!.:,;—“”0123456789’'))
    palabraspos = textopos.split()
    palabrasneg = textoneg.split()
    textop=""
    texton=""
    #rearmamos el texto debido a que existen carácteres especiales
    for w in palabraspos:
        textop=textop+w+' '
    for w in palabrasneg:
        texton=texton+w+' '
    return textop, texton

### Lectura de documentos

In [9]:
def lee_documento(filename='NULL',texto=''):
    if filename != 'NULL':
        textop, texton=cargar_datos(filename)
    textop = textop.lower()
    texton = texton.lower()
    #obtenemos el vocabulario
    tokensp=textop.split()
    vocabulariop=[]
    vocabularion=[]
    tokensn = texton.split()
    for t in tokensp:
        if(t not in vocabulariop):
            vocabulariop.append(t)
    for t in tokensn:
        if(t not in vocabularion):
            vocabularion.append(t)
    #variables de procesamiento
    distp={}
    distn={}
    frecn={}
    frecp={}
    frecp,distp=distribucion(tokensp,vocabulariop)
    frecn, distn = distribucion(tokensn, vocabularion)
    return frecp,distp,tokensp,vocabulariop,textop,frecn,distn,tokensn,vocabularion,texton

## Ejecución de algoritmos

In [11]:
#Lectura de documento para el corpus de suicidio
frecp,distp,tokensp,vocabulariop,textop,frecn,distn,tokensn,vocabularion,texton = lee_documento('/home/matias/Documentos/MGP/Proyecto_tesis/Dataset/SMS_DATA_ORIGINAL.csv')

In [12]:
print('Frecuencias:', len(frecp))
print("Distribuciones: ", len(distp))
print("Tokens: ", len(tokensp))
print('Vocabulario: ', len(vocabulariop))
print('Texto: ', len(textop))


Frecuencias: 8070
Distribuciones:  8070
Tokens:  101671
Vocabulario:  8070
Texto:  544467


In [13]:
print('Frecuencias:', len(frecn))
print("Distribuciones: ", len(distn))
print("Tokens: ", len(tokensn))
print('Vocabulario: ', len(vocabularion))
print('Texto: ', len(texton))

Frecuencias: 3349
Distribuciones:  3349
Tokens:  25052
Vocabulario:  3349
Texto:  130002


### Grado de Fractalidad

In [14]:
#ejecución de algoritmo Grado de Fractalidad
def grado_de_fractalidad(tokens,vocabulario,frec,dist,regresa_kw=False,regresa_df=True,top_n=np.inf,escribe_arch=False):
    frac_x=fractalidad(tokens,vocabulario,frec,dist)
    sorted_x = sorted(frac_x.items(), key=operator.itemgetter(1), reverse=True)
    # print('Time GF: '+str(elapsed_time))

    #Imprimir y guardar resultados de GF
    if regresa_df:
        if top_n != np.inf:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x[:top_n]]
        else:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x]
        #Ordenar resultados por medida combinada
        df.sort(key=lambda x: x[3],reverse=True)
        if regresa_kw==False:
            df = [dato[0] for dato in df]
            by_MC=pd.DataFrame(df, columns=['word'])
        else:
            by_MC=pd.DataFrame(df, columns=['word','frecuency','Degree_of_fractality','Combined_measure'])
        if escribe_arch:
            by_MC.to_csv('GF.csv')
    else:
        if top_n != np.inf:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x[:top_n]]
        else:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x]
        #Ordenar resultados por medida combinada
        df.sort(key=lambda x: x[3],reverse=True)
        if regresa_kw==False:
            by_MC = [dato[0] for dato in df]
        else:
            by_MC = df
        if escribe_arch:
            print('\nNo se tiene implementada la escritura de archivo cuando regresa_df==False\n')
    return by_MC

def use_gf(texto,regresa_kw=False,regresa_df=False,top_n=np.inf,escribe_arch=False):
    tokens=texto.split()
    vocabulario=[]
    for t in tokens:
        if(t not in vocabulario):
            vocabulario.append(t)
    #variables de procesamiento
    dist={}
    frec={}
    frec,dist=distribucion(tokens,vocabulario)
    df = grado_de_fractalidad(tokens,vocabulario,frec,dist,regresa_kw,regresa_df,top_n,escribe_arch)
    return df

fractPositivas = use_gf(textop,regresa_kw=True,regresa_df=True,top_n=1286)
fractNegativas = use_gf(texton,regresa_kw=True,regresa_df=True,top_n=1286)

In [15]:
fractPositivas

,word,frecuency,Degree_of_fractality,Combined_measure
0,caca,23,85221.880907,116049.007469
1,ok,12,78526.466331,84744.289783
2,capsulas,11,78681.018360,81937.836981
3,observaciones,19,63249.000502,80879.887149
4,calzoncillo,11,77619.128001,80831.992129
...,...,...,...,...
1281,sentiría,2,31522.006956,9489.069617
1282,impulsivo,2,31511.197263,9485.815576
1283,frecuentes,2,31441.375211,9464.797043
1284,actuó,2,31424.739678,9459.789249


In [16]:
fractNegativas

,word,frecuency,Degree_of_fractality,Combined_measure
0,ángel,10,18176.971038,18176.971038
1,tío,18,12706.403384,15949.998807
2,siento,118,6956.870176,14413.814144
3,vómitos,7,16922.938652,14301.542286
4,vida,82,7257.344789,13889.206989
...,...,...,...,...
1115,edad,2,1798.162775,541.300932
1116,grosero,2,1788.353395,538.348015
1117,seria,2,1772.230261,533.494468
1118,ayuden,2,1764.830483,531.266913


In [18]:
#Guarda los resultados del grado de fractalidad en un CSV. Siempre hay que cambiar la versión y comentar para no sobreescribir
fractPositivas.to_csv('/home/matias/Documentos/MGP/Proyecto_tesis/output/fractalidad_etiqueta5_EmoPro_prueba1.csv')
fractNegativas.to_csv('/home/matias/Documentos/MGP/Proyecto_tesis/output/fractalidad_etiqueta0_EmoPro_prueba1.csv')

### TextRank

In [19]:
#Cambio del máximo de tamaño de texto para que quepa el corpus (1.3 millones)
nlp.max_length = 1500000

In [20]:
#ejecución de algoritmo de TextRank
# start_time = time()
def use_TextRank(texto,regresa_kw=False,regresa_df=False,top_n=np.inf,escribe_arch=False):
    tr4w = TextRank4Keyword()
    tr4w.analyze(texto, candidate_pos = ['NOUN','PROPN'], window_size=4, lower=False)
    kwTR=tr4w.get_keywords(100)

    #Guardar resultados de TextRank
    if regresa_df:
        if top_n!=np.inf:
            if regresa_kw==True:
                salida = [[key, kwTR[key]] for key in kwTR.keys()][:top_n]
                dftr=pd.DataFrame(salida, columns=['word', 'Index'])
            else:
                salida = list(kwTR.keys())[:top_n]
                dftr=pd.DataFrame(salida, columns=['word'])
        else:
            if regresa_kw==True:
                salida = [[key, kwTR[key]] for key in kwTR.keys()]
                dftr=pd.DataFrame(salida, columns=['word', 'Index'])
            else:
                salida = list(kwTR.keys())
                dftr=pd.DataFrame(salida, columns=['word'])
    else:
        if top_n!=np.inf:
            if regresa_kw==True:
                dftr = [[key, kwTR[key]] for key in kwTR.keys()][:top_n]
            else:
                dftr = list(kwTR.keys())[:top_n]
        else:
            if regresa_kw==True:
                dftr = [[key, kwTR[key]] for key in kwTR.keys()]
            else:
                dftr = list(kwTR.keys())
        # elapsed_time = time() - start_time
        # print('Time TextRank: '+str(elapsed_time))
        if escribe_arch:
            dftr.to_csv('TextRank.csv')
    return dftr

trpos = use_TextRank(textop,top_n=1286)
trneg = use_TextRank(texton,top_n=1286)

In [21]:
trpos

['trabajo',
 'cosas',
 'casa',
 'semana',
 'tiempo',
 'vida',
 'ansiedad',
 'horas',
 'noche',
 'familia',
 'mañana',
 'ganas',
 'problemas',
 'madre',
 'cabeza',
 'situación',
 'sueño',
 'problema',
 'momento',
 'tratamiento',
 'miedo',
 'medicación',
 'sensación',
 'ánimo',
 'dolor',
 'hora',
 'gente',
 'general',
 'años',
 'marido',
 'mujer',
 'pareja',
 'persona',
 'gracias',
 'cosa',
 'padre',
 'momentos',
 'hija',
 'tema',
 'consulta',
 'cuestionario',
 'doctora',
 'forma',
 'cita',
 'mes',
 'vacaciones',
 'cambio',
 'comida',
 'médico',
 'intento',
 'tensión',
 'hijo',
 'hermana',
 'amigos',
 'semanas',
 'nervios',
 'pastillas',
 'hijos',
 'niño',
 'pensamientos',
 'falta',
 'baja',
 'sábado',
 'relación',
 'lunes',
 'rato',
 'amiga',
 'cambios',
 'dolores',
 'hospital',
 'año',
 'cansancio',
 'sigo',
 'estrés',
 'viernes',
 'cuerpo',
 'meses',
 'madrid',
 'discusión',
 'enfermedad',
 'domingo',
 'salud',
 'psiquiatra',
 'personas',
 'mañanas',
 'gusto',
 'preocupación',
 'test'

In [22]:
trneg

['vida',
 'ansiedad',
 'ganas',
 'cabeza',
 'dolor',
 'casa',
 'madre',
 'cosas',
 'noche',
 'tiempo',
 'mierda',
 'horas',
 'miedo',
 'problemas',
 'mundo',
 'cosa',
 'medicación',
 'semana',
 'cama',
 'trabajo',
 'persona',
 'mañana',
 'comida',
 'dolores',
 'años',
 'familia',
 'daño',
 'culpa',
 'situación',
 'pena',
 'hospital',
 'fuerzas',
 'hija',
 'año',
 'gente',
 'momento',
 'padre',
 'hora',
 'cuerpo',
 'nervios',
 'voces',
 'personas',
 'problema',
 'cita',
 'gracias',
 'sueño',
 'pesadillas',
 'médico',
 'tío',
 'cambio',
 'sigo',
 'atracones',
 'tensión',
 'mente',
 'hijos',
 'discusiones',
 'pastillas',
 'caso',
 'sensación',
 'asco',
 'hermano',
 'amiga',
 'chocolate',
 'doctora',
 'pensamientos',
 'consulta',
 'decisión',
 'pruebas',
 'resto',
 'calle',
 'discusión',
 'maría',
 'baja',
 'sentido',
 'ángel',
 'jueves',
 'mujer',
 'pareja',
 'domingo',
 'noches',
 'amigos',
 'enfermedad',
 'lorazepam',
 'falta',
 'coche',
 'tipo',
 'ayuda',
 'sábado',
 'peso',
 'segurida

In [45]:
len(trneg)

102

In [23]:
#Guarda los resultados del TextRank en un txt. Siempre hay que cambiar la versión y comentar para no sobreescribir
with open('/home/matias/Documentos/MGP/Proyecto_tesis/output/TextRank_resultados_etiqueta5_EmoPro_prueba1.txt','w') as output:
    output.write(str(trpos))
with open('//home/matias/Documentos/MGP/Proyecto_tesis/output/TextRank_resultado_setiqueta0_EmoPro_prueba1.txt','w') as output:
    output.write(str(trneg))

## Comparación de los resultados de fractalidad y TextRank con el conjunto de Palabras Prototípicas

In [24]:
with open('/home/matias/Documentos/MGP/Proyecto_tesis/Palabras_proto/EmoPro-Dataset.csv', encoding="utf-8-sig") as f:
        palabrasProto = pd.read_csv(f)
        

In [25]:
palabrasProto

,Word,Few_Raters,Prototypicality_Mean,Prototypicality_SD,Prototypicality_NRaters,Prototypicality_%Raters,Valence_Mean,Valence_SD,Valence_NRaters,Valence_%Raters,...,Concreteness_%Raters,Concreteness_Source,Zipf_EsPal,POS,Emotionality,Family,Very_related_emotions,Pure_word,Dominant_emotion,Emotional_exclusivity
0,abandonado,NaN,2.35,1.27,20,1.00,1.93,1.12,20,1.00,...,1.0,NEW,4.36,ADJECTIVE,3.07,abandonar,2.0,0.0,sadness,0.396
1,abandonarse,NaN,2.00,1.10,21,1.00,1.89,0.88,20,0.95,...,1.0,NEW,2.98,VERB,3.11,abandonar,2.0,0.0,sadness,0.326
2,abandono,NaN,2.38,1.50,21,1.00,2.30,1.22,20,1.00,...,1.0,NEW,4.31,NOUN,2.70,abandonar,3.0,0.0,sadness,0.350
3,abatido,NaN,3.37,1.42,20,0.95,2.77,1.75,26,1.00,...,1.0,NEW,3.59,ADJECTIVE,2.23,abatir,1.0,1.0,sadness,0.379
4,abatimiento,NaN,3.00,1.37,20,0.90,2.80,1.32,20,1.00,...,1.0,NEW,3.28,NOUN,2.20,abatir,1.0,1.0,sadness,0.340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,vital,NaN,2.09,1.38,22,1.00,6.35,1.53,20,1.00,...,1.0,NEW,4.50,ADJECTIVE,1.35,vital,0.0,0.0,happiness,0.706
1282,vitalidad,NaN,2.05,1.10,20,1.00,8.20,0.95,20,1.00,...,NaN,Duchon2013,3.74,NOUN,3.20,vital,1.0,1.0,happiness,0.901
1283,vitalizar,NaN,1.89,1.10,20,0.95,6.80,1.20,20,1.00,...,1.0,NEW,2.03,VERB,1.80,vital,1.0,1.0,happiness,0.741
1284,vulnerabilidad,NaN,2.29,1.45,21,1.00,2.30,1.08,20,1.00,...,1.0,NEW,3.49,NOUN,2.70,vulnerable,1.0,1.0,sadness,0.288


### Función para crear la medida de Jaccard

In [28]:
#Medida Jaccard que mide la proporción de elementos de un conjunto en otro
def jaccard(s1: set, s2: set):
    return len(s1 & s2) / len(s1 | s2)   

In [29]:
def coinsidencia(conjuntoA, conjuntoB):
    matches = []
    for palabra in conjuntoA:
        if palabra in conjuntoB:
            matches.append(palabra)
    conteo = len(matches)
    return matches, conteo

In [30]:
jaccFractalidadPos = jaccard(set(palabrasProto['Word']), set(fractPositivas['word']))
jaccTRPos = jaccard(set(palabrasProto['Word']), set(trpos))
jaccFractalidadNeg = jaccard(set(palabrasProto['Word']), set(fractNegativas['word']))
jaccTRNeg = jaccard(set(palabrasProto['Word']), set(trneg))
# _, conteosTR = jaccard(texto, dftr)
# _, conteosFract= jaccard(texto, list(df1['word']))
# _, conteosProto= jaccard(texto, list(palabrasProto['Palabra']))

print('Jaccard de fractalidad positivas: ', jaccFractalidadPos)

print('Jaccard de TextRank positivas: ', jaccTRPos)


print('Jaccard de fractalidad negativas: ', jaccFractalidadNeg)

print('Jaccard de TextRank negativas: ', jaccTRNeg)


Jaccard de fractalidad positivas:  0.0362610797743755
Jaccard de TextRank positivas:  0.00725689404934688
Jaccard de fractalidad negativas:  0.04155844155844156
Jaccard de TextRank negativas:  0.00725689404934688


In [31]:
coinTRPos, conteoTRPos= coinsidencia(list(palabrasProto['Word']), trpos)
coinTRNeg, conteoTRNeg = coinsidencia(list(palabrasProto['Word']), trneg)
coinFracPos, conteoFracPos = coinsidencia(list(palabrasProto['Word']), list(fractPositivas['word']))
coinFracNeg, conteoFracNeg = coinsidencia(list(palabrasProto['Word']), list(fractNegativas['word']))

In [32]:
print('Conteo TR positivas:', conteoTRPos)
print('Conteo TR negativas:', conteoTRNeg)
print('Conteo Fractalidad positivas:', conteoFracPos)
print('Conteo Fractalidad negativas:', conteoFracNeg)

Conteo TR positivas: 10
Conteo TR negativas: 10
Conteo Fractalidad positivas: 90
Conteo Fractalidad negativas: 96


### Altamente prototípicas

Esta sección aprovecha los resultados de los algoritmos para revisar la coinsidencia entre los resultados de éstos, y de las palabras altamente prototípicas. 

El procedimiento es el siguiente:
- Selección de las palabras altamente prototípicas del EmoPro (Media de prototipicalidad >= 3.0)
- Ajuste de palabras de etiqueta 0 de fractalidad para que el conjunto sea del mismo tamaño que el de AP
- Ajuste de palabras de etiqueta 5 de fractalidad para que el conjunto sea del mismo tamaño que el de AP
- Comparación de coinsidencias y medida Jaccard

In [33]:
palabrasProto

,Word,Few_Raters,Prototypicality_Mean,Prototypicality_SD,Prototypicality_NRaters,Prototypicality_%Raters,Valence_Mean,Valence_SD,Valence_NRaters,Valence_%Raters,...,Concreteness_%Raters,Concreteness_Source,Zipf_EsPal,POS,Emotionality,Family,Very_related_emotions,Pure_word,Dominant_emotion,Emotional_exclusivity
0,abandonado,NaN,2.35,1.27,20,1.00,1.93,1.12,20,1.00,...,1.0,NEW,4.36,ADJECTIVE,3.07,abandonar,2.0,0.0,sadness,0.396
1,abandonarse,NaN,2.00,1.10,21,1.00,1.89,0.88,20,0.95,...,1.0,NEW,2.98,VERB,3.11,abandonar,2.0,0.0,sadness,0.326
2,abandono,NaN,2.38,1.50,21,1.00,2.30,1.22,20,1.00,...,1.0,NEW,4.31,NOUN,2.70,abandonar,3.0,0.0,sadness,0.350
3,abatido,NaN,3.37,1.42,20,0.95,2.77,1.75,26,1.00,...,1.0,NEW,3.59,ADJECTIVE,2.23,abatir,1.0,1.0,sadness,0.379
4,abatimiento,NaN,3.00,1.37,20,0.90,2.80,1.32,20,1.00,...,1.0,NEW,3.28,NOUN,2.20,abatir,1.0,1.0,sadness,0.340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,vital,NaN,2.09,1.38,22,1.00,6.35,1.53,20,1.00,...,1.0,NEW,4.50,ADJECTIVE,1.35,vital,0.0,0.0,happiness,0.706
1282,vitalidad,NaN,2.05,1.10,20,1.00,8.20,0.95,20,1.00,...,NaN,Duchon2013,3.74,NOUN,3.20,vital,1.0,1.0,happiness,0.901
1283,vitalizar,NaN,1.89,1.10,20,0.95,6.80,1.20,20,1.00,...,1.0,NEW,2.03,VERB,1.80,vital,1.0,1.0,happiness,0.741
1284,vulnerabilidad,NaN,2.29,1.45,21,1.00,2.30,1.08,20,1.00,...,1.0,NEW,3.49,NOUN,2.70,vulnerable,1.0,1.0,sadness,0.288


In [ ]:
#Selección de altamente prototípias de EmoPro
altaProto = palabrasProto[palabrasProto['Prototypicality_Mean']>= 3]

In [36]:
altaProto 

,Word,Few_Raters,Prototypicality_Mean,Prototypicality_SD,Prototypicality_NRaters,Prototypicality_%Raters,Valence_Mean,Valence_SD,Valence_NRaters,Valence_%Raters,...,Concreteness_%Raters,Concreteness_Source,Zipf_EsPal,POS,Emotionality,Family,Very_related_emotions,Pure_word,Dominant_emotion,Emotional_exclusivity
3,abatido,NaN,3.37,1.42,20,0.95,2.77,1.75,26,1.00,...,1.00,NEW,3.59,ADJECTIVE,2.23,abatir,1.0,1.0,sadness,0.379
4,abatimiento,NaN,3.00,1.37,20,0.90,2.80,1.32,20,1.00,...,1.00,NEW,3.28,NOUN,2.20,abatir,1.0,1.0,sadness,0.340
6,abochornado,NaN,3.90,1.37,21,0.95,2.45,1.28,20,1.00,...,0.95,NEW,2.24,ADJECTIVE,2.55,bochorno,2.0,0.0,anger,0.264
7,abochornarse,NaN,3.84,1.54,21,0.90,3.44,2.18,26,0.96,...,0.95,NEW,0.51,VERB,1.56,bochorno,0.0,0.0,anger,0.248
13,abrumado,NaN,3.24,1.48,22,0.95,3.90,1.48,20,1.00,...,1.00,NEW,3.24,ADJECTIVE,1.10,abrumar,0.0,0.0,sadness,0.269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,vergonzoso,NaN,3.60,1.31,20,1.00,2.55,1.28,20,1.00,...,1.00,NEW,3.57,ADJECTIVE,2.45,vergüenza,0.0,0.0,sadness,0.294
1271,vergüenza,NaN,4.38,0.97,21,1.00,3.85,1.18,20,1.00,...,1.00,NEW,4.37,NOUN,1.15,vergüenza,1.0,1.0,fear,0.260
1273,victorioso,NaN,3.70,1.13,20,1.00,7.05,1.47,20,1.00,...,NaN,Hinojosa2016_BRM,3.55,ADJECTIVE,2.05,victoria,1.0,1.0,happiness,0.813
1279,violentarse,NaN,3.05,1.47,21,0.90,1.80,1.11,20,1.00,...,0.95,NEW,1.63,VERB,3.20,violento,3.0,0.0,anger,0.295


In [ ]:
#Igualar número con resultados de fractalidad e0
fractPositivas = fractPositivas[:549]

In [39]:
fractPositivas

,word,frecuency,Degree_of_fractality,Combined_measure
0,caca,23,85221.880907,116049.007469
1,ok,12,78526.466331,84744.289783
2,capsulas,11,78681.018360,81937.836981
3,observaciones,19,63249.000502,80879.887149
4,calzoncillo,11,77619.128001,80831.992129
...,...,...,...,...
544,muscular,5,36518.004889,25524.990036
545,pasos,5,36510.492729,25519.739261
546,prescripción,6,32758.927861,25491.400677
547,quiste,3,53404.728254,25480.530953


In [40]:
#Igualar número con resultados de fractalidad e5
fractNegativas = fractNegativas[:549]

In [41]:
fractNegativas

,word,frecuency,Degree_of_fractality,Combined_measure
0,ángel,10,18176.971038,18176.971038
1,tío,18,12706.403384,15949.998807
2,siento,118,6956.870176,14413.814144
3,vómitos,7,16922.938652,14301.542286
4,vida,82,7257.344789,13889.206989
...,...,...,...,...
544,quiera,3,7920.344064,3778.964498
545,cabreado,3,7907.603557,3772.885731
546,padres,5,5378.794742,3759.616184
547,quede,3,7852.723224,3746.701158


In [42]:
trpos = trpos[:549]

In [46]:
jaccFractalidadPos = jaccard(set(palabrasProto['Word']), set(fractPositivas['word']))
jaccTRPos = jaccard(set(palabrasProto['Word']), set(trpos))
jaccFractalidadNeg = jaccard(set(palabrasProto['Word']), set(fractNegativas['word']))
jaccTRNeg = jaccard(set(palabrasProto['Word']), set(trneg))
# _, conteosTR = jaccard(texto, dftr)
# _, conteosFract= jaccard(texto, list(df1['word']))
# _, conteosProto= jaccard(texto, list(palabrasProto['Palabra']))

print('Jaccard de fractalidad positivas: ', jaccFractalidadPos)

print('Jaccard de TextRank positivas: ', jaccTRPos)


print('Jaccard de fractalidad negativas: ', jaccFractalidadNeg)

print('Jaccard de TextRank negativas: ', jaccTRNeg)


Jaccard de fractalidad positivas:  0.025139664804469275
Jaccard de TextRank positivas:  0.00725689404934688
Jaccard de fractalidad negativas:  0.028011204481792718
Jaccard de TextRank negativas:  0.00725689404934688


In [47]:
coinTRPos, conteoTRPos= coinsidencia(list(palabrasProto['Word']), trpos)
coinTRNeg, conteoTRNeg = coinsidencia(list(palabrasProto['Word']), trneg)
coinFracPos, conteoFracPos = coinsidencia(list(palabrasProto['Word']), list(fractPositivas['word']))
coinFracNeg, conteoFracNeg = coinsidencia(list(palabrasProto['Word']), list(fractNegativas['word']))

In [48]:
print('Conteo TR positivas:', conteoTRPos)
print('Conteo TR negativas:', conteoTRNeg)
print('Conteo Fractalidad positivas:', conteoFracPos)
print('Conteo Fractalidad negativas:', conteoFracNeg)

Conteo TR positivas: 10
Conteo TR negativas: 10
Conteo Fractalidad positivas: 45
Conteo Fractalidad negativas: 50


TextRak, a pesar de que se le solicitaron 1286 y 549 palabras clave, sólo regresó 102